In [1]:
# I will implement a basic ResNet model for image classification.
# The model will consist of several convolutional layers, batch normalization, and ReLU activations.
# The architecture will include skip connections to allow gradients to flow more easily through the network.
# The model will be built using PyTorch.
# The dataset used will have 10 classes, and the input images will need to be resized to 200x200 pixels.
# The model will be trained using the Adam optimizer and cross-entropy loss.
# data augmentation will be applied later. 

In [ ]:
#take in images and crop images to 200x200 pixels
